# Parsing WARC File

### 🛠 Installing Necessary Libraries

In [1]:
pip install warcio beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
from warcio.archiveiterator import ArchiveIterator
from bs4 import BeautifulSoup
import sqlite3
import time

def process_warc_file_in_chunks(warc_path, db_path, batch_size=100):
    """
    WARC dosyasını parça parça işleyerek SQLite'a kaydeder
    
    Args:
        warc_path: WARC dosyasının yolu
        db_path: SQLite veritabanının yolu
        batch_size: Her defasında işlenecek kayıt sayısı
    """
    # SQLite bağlantısını oluştur
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    # Tablo oluştur (eğer yoksa)
    c.execute('CREATE TABLE IF NOT EXISTS pages (id INTEGER PRIMARY KEY, url TEXT, title TEXT, html TEXT)')
    
    # İşleme istatistikleri
    start_time = time.time()
    total_records = 0
    batch_records = []
    
    # WARC dosyasını aç ve kayıtları işle
    with open(warc_path, 'rb') as stream:
        for record in ArchiveIterator(stream):
            if record.rec_type == 'response':
                try:
                    url = record.rec_headers.get_header('WARC-Target-URI')
                    raw_html = record.content_stream().read().decode('utf-8', errors='replace')
                    
                    soup = BeautifulSoup(raw_html, 'html.parser')
                    title = soup.title.string if soup.title else ''
                    
                    batch_records.append((url, title, raw_html))
                    total_records += 1
                    
                    # Belirli bir sayıda kayıt toplandıysa veritabanına kaydet
                    if len(batch_records) >= batch_size:
                        c.executemany('INSERT INTO pages (url, title, html) VALUES (?, ?, ?)', batch_records)
                        conn.commit()
                        
                        elapsed_time = time.time() - start_time
                        print(f"İşlenen kayıt: {total_records}, Geçen süre: {elapsed_time:.2f} saniye")
                        
                        # Belleği temizle
                        batch_records = []
                
                except Exception as e:
                    print(f"Hata oluştu: {e} - URL: {url if 'url' in locals() else 'bilinmiyor'}")
    
    # Kalan kayıtları kaydet
    if batch_records:
        c.executemany('INSERT INTO pages (url, title, html) VALUES (?, ?, ?)', batch_records)
        conn.commit()
    
    # İstatistikleri göster
    total_time = time.time() - start_time
    print(f"Toplam işlenen kayıt: {total_records}")
    print(f"Toplam geçen süre: {total_time:.2f} saniye")
    print(f"Saniyede ortalama kayıt: {total_records/total_time:.2f}")
    
    conn.close()

# Kodu çalıştır
warc_file = "./website_data.warc"
db_file = "websites.db"
process_warc_file_in_chunks(warc_file, db_file, batch_size=500)

İşlenen kayıt: 500, Geçen süre: 49.11 saniye
İşlenen kayıt: 1000, Geçen süre: 92.11 saniye
Hata oluştu: The markup you provided was rejected by the parser. Trying a different parser or a different encoding may help.

Original exception(s) from parser:
 AssertionError: expected name token at '<![��P���a�9\x02l~Q����' - URL: https://files.dainikshiksha.com/136301/conversions/najirpur-thumb.webp
İşlenen kayıt: 1500, Geçen süre: 139.04 saniye
İşlenen kayıt: 2000, Geçen süre: 181.75 saniye
İşlenen kayıt: 2500, Geçen süre: 208.13 saniye
İşlenen kayıt: 3000, Geçen süre: 232.13 saniye
İşlenen kayıt: 3500, Geçen süre: 254.73 saniye
İşlenen kayıt: 4000, Geçen süre: 278.45 saniye
İşlenen kayıt: 4500, Geçen süre: 300.51 saniye
İşlenen kayıt: 5000, Geçen süre: 322.71 saniye
İşlenen kayıt: 5500, Geçen süre: 365.81 saniye
İşlenen kayıt: 6000, Geçen süre: 410.47 saniye
İşlenen kayıt: 6500, Geçen süre: 450.23 saniye
İşlenen kayıt: 7000, Geçen süre: 496.43 saniye
İşlenen kayıt: 7500, Geçen süre: 527.93 

/opt/anaconda3/lib/python3.12/html/parser.py:171: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


İşlenen kayıt: 8500, Geçen süre: 584.87 saniye
İşlenen kayıt: 9000, Geçen süre: 612.02 saniye
İşlenen kayıt: 9500, Geçen süre: 660.96 saniye
İşlenen kayıt: 10000, Geçen süre: 692.42 saniye
İşlenen kayıt: 10500, Geçen süre: 717.93 saniye
İşlenen kayıt: 11000, Geçen süre: 740.11 saniye
İşlenen kayıt: 11500, Geçen süre: 756.56 saniye
İşlenen kayıt: 12000, Geçen süre: 769.75 saniye
İşlenen kayıt: 12500, Geçen süre: 783.53 saniye
İşlenen kayıt: 13000, Geçen süre: 798.41 saniye
İşlenen kayıt: 13500, Geçen süre: 818.17 saniye
İşlenen kayıt: 14000, Geçen süre: 836.72 saniye
İşlenen kayıt: 14500, Geçen süre: 856.90 saniye
İşlenen kayıt: 15000, Geçen süre: 874.91 saniye
İşlenen kayıt: 15500, Geçen süre: 891.80 saniye
İşlenen kayıt: 16000, Geçen süre: 906.40 saniye
İşlenen kayıt: 16500, Geçen süre: 926.27 saniye
İşlenen kayıt: 17000, Geçen süre: 945.42 saniye
İşlenen kayıt: 17500, Geçen süre: 961.74 saniye
İşlenen kayıt: 18000, Geçen süre: 978.91 saniye
İşlenen kayıt: 18500, Geçen süre: 996.68 sa